# Analisis de sentimientos

## Carga de librerias

In [7]:
import os
from string import punctuation

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd

from src.features.build_features import processing_text_pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/isbelloq/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Variables globables

In [8]:
# Semilla para asegurar replicabilidad
SEED = 123

# Definicion de rutas
PROJECT_PATH = os.path.join(os.environ['HOME'], 'retos/20220917_sentimientos')
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
RAW_PATH = os.path.join(DATA_PATH, 'raw')
RAW_PATH = os.path.join(DATA_PATH, 'raw')
RAW_PATH = os.path.join(DATA_PATH, 'raw')

## Lectura de datos

In [9]:
df = pd.read_csv(os.path.join(RAW_PATH, 'tweets_public.csv'))

## Preprocesamiento de informacion

In [10]:
# Simbolos de puntuacion
spanish_punctuation = ''.join([i for i in punctuation if i != '#'])
spanish_punctuation += '…¡¿€'
# Stopwords
spanish_stopwords = stopwords.words('spanish')
spanish_stopwords.remove('no')

In [15]:
# Seleccino de columnas de interes
prepros_df = df.loc[:,['text', 'airline_sentiment']]
prepros_df = processing_text_pipeline(prepros_df, 'text', spanish_punctuation, spanish_stopwords)

,text,airline_sentiment,pre_text
0,Trabajar en #Ryanair como #TMA: https://t.co/r...,neutral,trabajar #ryanair #tma #empleo
1,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,neutral,gusten cancun viaja disfruta manera igual
2,Sabiais que @Iberia te trata muy bien en santi...,negative,sabiais trata bien santiago chile cambia asien...
3,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,negative,nunca nunca nunca pidais cafe ryanair bueno ve...
4,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,positive,exito
...,...,...,...
7862,@Iberia @iberiaexpress especialistas en dejart...,negative,especialistas dejarte tirado navidad :crying_f...
7863,"Con @Iberia, mi destino a un solo click. ¡Dese...",neutral,destino solo click deseadme suerte #holacolomb...
7864,@Iberia Muy bien. Muchas gracias,positive,bien muchas gracias
7865,Es que volar con Ryanair es peor que irte a ch...,negative,volar ryanair peor irte chingar madre culpa pobre


In [34]:
corpus = ' '.join(prepros_df['pre_text'])

In [36]:
FreqDist(['a', 'b','a','a','b','c','a'])

FreqDist({'a': 4, 'b': 2, 'c': 1})

In [38]:
FreqDist('a b a a b c a')

FreqDist({' ': 6, 'a': 4, 'b': 2, 'c': 1})

In [40]:
from nltk import FreqDist, ngrams

for igram in range(2):
    
    print(i+1)

1
2


In [47]:
help(FreqDist)

Help on class FreqDist in module nltk.probability:

class FreqDist(collections.Counter)
 |  FreqDist(samples=None)
 |  
 |  A frequency distribution for the outcomes of an experiment.  A
 |  frequency distribution records the number of times each outcome of
 |  an experiment has occurred.  For example, a frequency distribution
 |  could be used to record the frequency of each word type in a
 |  document.  Formally, a frequency distribution can be defined as a
 |  function mapping from each sample to the number of times that
 |  sample occurred as an outcome.
 |  
 |  Frequency distributions are generally constructed by running a
 |  number of experiments, and incrementing the count for a sample
 |  every time it is an outcome of an experiment.  For example, the
 |  following code will produce a frequency distribution that encodes
 |  how often each word occurs in a text:
 |  
 |      >>> from nltk.tokenize import word_tokenize
 |      >>> from nltk.probability import FreqDist
 |      >

In [108]:
foo = ngrams(corpus.split(), 2)
bar =  FreqDist(corpus.split())
type(bar)

nltk.probability.FreqDist

In [92]:
for i in foo:
    print(i)

('trabajar', '#ryanair')
('#ryanair', '#tma')
('#tma', '#empleo')
('#empleo', 'gusten')
('gusten', 'cancun')
('cancun', 'viaja')
('viaja', 'disfruta')
('disfruta', 'manera')
('manera', 'igual')
('igual', 'sabiais')
('sabiais', 'trata')
('trata', 'bien')
('bien', 'santiago')
('santiago', 'chile')
('chile', 'cambia')
('cambia', 'asiento')
('asiento', 'manda')
('manda', 'volar')
('volar', 'wc')
('wc', 'trasero')
('trasero', 'nunca')
('nunca', 'nunca')
('nunca', 'nunca')
('nunca', 'pidais')
('pidais', 'cafe')
('cafe', 'ryanair')
('ryanair', 'bueno')
('bueno', 'vendan')
('vendan', 'bordo')
('bordo', 'exito')
('exito', 'wow')
('wow', 'bonito')
('bonito', 'solo')
('solo', 'volado')
('volado', 'unas')
('unas', 'veces')
('veces', 'siempre')
('siempre', 'tierra')
('tierra', 'iberia')
('iberia', 'plus')
('plus', 'cumple')
('cumple', 'años')
('años', 'queremos')
('queremos', 'celebrarlo')
('celebrarlo', 'contigo')
('contigo', 'manera')
('manera', 'especial')
('especial', 'elige')
('elige', 'numero

In [89]:
bar

FreqDist({'iberia': 1802, 'no': 1642, 'ryanair': 1481, 'vuelo': 905, 'hola': 571, 'solo': 503, 'si': 459, 'vuelos': 447, 'madrid': 423, 'gracias': 421, ...})

In [111]:
foo_2 = pd.DataFrame.from_dict(bar, orient='index')
foo_2.sort_values(by = 0, ascending = False)

,0
iberia,1802
no,1642
ryanair,1481
vuelo,905
hola,571
...,...
porteña,1
antidoto,1
veneno,1
muerdas,1


In [82]:
bar.items()

dict_items([(('trabajar',), 16), (('#ryanair',), 107), (('#tma',), 2), (('#empleo',), 20), (('gusten',), 2), (('cancun',), 7), (('viaja',), 18), (('disfruta',), 2), (('manera',), 103), (('igual',), 51), (('sabiais',), 1), (('trata',), 10), (('bien',), 90), (('santiago',), 18), (('chile',), 19), (('cambia',), 7), (('asiento',), 60), (('manda',), 8), (('volar',), 136), (('wc',), 1), (('trasero',), 1), (('nunca',), 82), (('pidais',), 2), (('cafe',), 6), (('ryanair',), 1481), (('bueno',), 40), (('vendan',), 1), (('bordo',), 28), (('exito',), 4), (('wow',), 4), (('bonito',), 7), (('solo',), 503), (('volado',), 16), (('unas',), 25), (('veces',), 65), (('siempre',), 119), (('tierra',), 90), (('iberia',), 1802), (('plus',), 117), (('cumple',), 92), (('años',), 235), (('queremos',), 90), (('celebrarlo',), 80), (('contigo',), 84), (('especial',), 89), (('elige',), 81), (('numero',), 137), (('favorito',), 82), (('barometro',), 10), (('#business',), 12), (('vueling',), 103), (('compañias',), 46), 

In [54]:
help(nltk.probability.FreqDist)

Help on class FreqDist in module nltk.probability:

class FreqDist(collections.Counter)
 |  FreqDist(samples=None)
 |  
 |  A frequency distribution for the outcomes of an experiment.  A
 |  frequency distribution records the number of times each outcome of
 |  an experiment has occurred.  For example, a frequency distribution
 |  could be used to record the frequency of each word type in a
 |  document.  Formally, a frequency distribution can be defined as a
 |  function mapping from each sample to the number of times that
 |  sample occurred as an outcome.
 |  
 |  Frequency distributions are generally constructed by running a
 |  number of experiments, and incrementing the count for a sample
 |  every time it is an outcome of an experiment.  For example, the
 |  following code will produce a frequency distribution that encodes
 |  how often each word occurs in a text:
 |  
 |      >>> from nltk.tokenize import word_tokenize
 |      >>> from nltk.probability import FreqDist
 |      >

In [ ]:
thisset = {"apple", "banana", "cherry"}

thisset.add("banana")

print(thisset)

In [ ]:
foo[65]aa

In [ ]:
help(re.sub)

In [ ]:
prepros_df['text'].apply()


In [ ]:
punctuation

In [ ]:

punctuation

In [ ]:
punctuation = ''.join([i for i in punctuation if i != '#'])


In [ ]:
help(re.match)

In [ ]:
prepros_df.loc[1, 'text']

In [ ]:
# Limpieza de texto

## Elimincacion de links